<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jihoyeo/mobility-simulation-book/blob/main/ko/chapter2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>
<br>

## 2.3 Travel Time Data (Taxi Trip Record)

- 지금까지는 통행량 데이터를 살펴보았습니다. 이 장에서는 통행시간 데이터를 살펴봅니다.
- 스마트카드(지하철,버스), 택시 탑승 및 호출 이력, 스마트폰 GPS 데이터 등 다양한 경로를 통해 여러분들의 이동 데이터가 수집되고 있습니다.
- 그 중 택시 데이터는 승차와 하차의 정확한 위치와 시간을 알 수 있다는 점에서 데이터의 신뢰성이 높습니다.
- 다양한 국가에서 이미 표준화된 형태의 택시 승하차 데이터를 공개하고 있습니다.
    - [Ney York City Taxi Trip Duration Data](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data)
    - [Chicago Taxi Trips data](https://data.cityofchicago.org/Transportation/Taxi-Trips-2013-2023-/wrvz-psew/about_data)
- 본 장에서는 서울의 택시 승하차 이력 데이터를 살펴보겠습니다. 다만, 국내의 경우 택시승하차 데이터가 오픈되어있지 않으므로, 여기선 가상의 데이터를 활용합니다.
    - 본 실습 데이터는 실제 데이터가 아니며, 분석을 위해 랜덤하게 생생된 1일치의 데이터입니다. 
    - 국내도 데이터 개방이 더 활성화 되어, 미국의 주요도시처럼 택시 및 스마트카드 데이터가 공개되는 날이 오기를 희망합니다.

### 2.3.1 데이터 읽기

In [3]:
import pandas as pd
import os
from datetime import datetime, timedelta

In [4]:
# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

# 한글 폰트 설정 (Windows 환경)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False  # 마이너스 기호 깨짐 방지

# seaborn 스타일 적용
sns.set(font='Malgun Gothic', rc={"axes.unicode_minus": False})

# 그래프 크기 기본 설정
plt.rcParams['figure.figsize'] = (10, 6)

In [5]:
def download_and_read_parquet(file_id, output_path="../data/chp2_tx_data_generated.parquet"):
    try:
        # Google Drive에서 파일 다운로드
        gdown.download(id=file_id, output=output_path, quiet=False)
        
        # Parquet 파일을 DataFrame으로 읽기
        df = pd.read_parquet(output_path)
        
        # 임시 파일 삭제 (데이터 용량이 매우 큰 경우 사용)
        # os.remove(output_path) # 다운로드 받은 데이터를 삭제하고 싶지 않을 때는 해당 라인을 주석처리
        
        return df
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [6]:
# 파일 불러오기
file_id = "1uJj-C_7pTkcRk8p5lYah-9Fp660OvS4b" # 구글 드라이브에 업로드 된 파일의 ID
# tx_data = download_and_read_parquet(file_id)
tx_data = pd.read_parquet('../data/chp2_tx_data_generated.parquet')

- 데이터의 시간 형식이 숫자형태로 되어있다. 이렇게 되어있을 경우 보기가 불편하고, 시간단위 연산이 어렵기 때문에 시간 타입의 데이터로 변환해주는 것이 좋다

In [7]:
# 시간 데이터 형식 변횐
tx_data['RIDE_DTIME'] = pd.to_datetime(tx_data['RIDE_DTIME'], format="%Y%m%d%H%M%S")
tx_data['ALIGHT_DTIME'] = pd.to_datetime(tx_data['ALIGHT_DTIME'], format="%Y%m%d%H%M%S")

# 통행시간 계산
tx_data['TRAVEL_TIME'] = (tx_data['ALIGHT_DTIME'] - tx_data['RIDE_DTIME']) / pd.Timedelta(minutes=1)

In [8]:
tx_data

,RIDE_DTIME,RIDE_POS_X,RIDE_POS_Y,ALIGHT_DTIME,ALIGHT_POS_X,ALIGHT_POS_Y,TRAVEL_TIME
0,2022-05-01 00:00:00,126.886907,37.482125,2022-05-01 00:07:04,126.896551,37.470549,7.066667
1,2022-05-01 00:00:00,126.960889,37.507369,2022-05-01 00:19:35,127.113011,37.503858,19.583333
2,2022-05-01 00:00:00,127.101762,37.467082,2022-05-01 00:13:08,127.052796,37.540526,13.133333
3,2022-05-01 00:00:00,127.055634,37.589778,2022-05-01 00:10:50,127.087957,37.597224,10.833333
4,2022-05-01 00:00:00,127.044648,37.567578,2022-05-01 00:03:57,127.037768,37.561673,3.950000
...,...,...,...,...,...,...,...
597090,2022-05-01 23:59:58,126.908084,37.515114,2022-05-02 00:30:52,126.781716,37.486141,30.900000
597091,2022-05-01 23:59:59,127.001369,37.558880,2022-05-02 00:14:48,127.034217,37.508548,14.816667
597092,2022-05-01 23:59:59,127.002711,37.505298,2022-05-02 00:05:59,127.011192,37.492397,6.000000
597093,2022-05-01 23:59:59,127.131851,37.535969,2022-05-02 00:09:24,127.160008,37.550329,9.416667


### 2.3.2 Basic Visualization

- 통행량 데이터와 마찬가지로 간단한 시각화를 해보자.
- 다음과 같은 오픈소스를 활용한다면 매끄럽게 시각화가 가능하다.
    - https://kepler.gl/
    - 데이터를 csv로 저장한 후, Kepler를 통해 시각화를 해보자.
    - [예시](https://kepler.gl/demo/map?mapUrl=https://dl.dropboxusercontent.com/scl/fi/8k0y58t4rceqvslmvhwyl/keplergl_7sjanmo.json?rlkey=8d6d24vw7l63t2bsch5cd53m9&dl=0)
- 통행시간의 경우 출발지-목적지(O-D) 단위로 통행시간을 시각화 하는 연습도 해보자.
    - O-D 단위의 시각화의 경우 무수히 많은 O-D pairs가 존재하기 때문에 시각적으로 표현하기가 매우 어렵다.
    - 또한, 지금 데이터의 공간단위가 이전에 다뤘던 행정구역과 같이 합산된 공간단위가 아니라, 세밀한 위경도 좌표로 표현되어 있기 때문에 더 어렵다.
    - 위와 같은 문제들을 어떻게 해결할 수 있을까? 

In [ ]:
tx_data.to_csv('../data/tx_data_generated.csv')

### 2.3.3 Exercise

GPT와 같은 AI 툴을 적극 활용해 문제를 풀어봅시다. 다만, 출력된 코드를 보고 이해하고, 코드가 실행되지 않는다면 어디서 실행되지 않는지 파악하고 올바르게 수정할 수 있어야 합니다.

#### [1] 통행 시간 데이터 분석 및 시각화

> 2.3 Section에서 불러온 가상의 택시 통행 데이터를 가지고 아래와 같은 분석을 수행해보자.

1. 택시 통행 데이터의 각 행에, 출발지역의 행정동 코드 & 도착지역의 행정동 코드를 매핑해봅시다. 이후 아래문제를 풀어보세요. `geopandas` 패키지에 있는  `sjoin` 함수를 사용하면 됩니다. 
2. 청담동에서 출발해서 연희동 도착하는 택시 통행만을 추출하여, 출발 시간대별(1시간단위로 합산) 통행시간의 변화를 Line Graph로 그려봅시다. 언제 가장 시간이 오래 걸리나요?  